<a href="https://colab.research.google.com/github/gabrielhenriquess/TelecomX/blob/main/Analise_TelecomX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📊 Análise de Evasão churn de Clientes - TelecomX


#📌 Extracão

In [38]:

import pandas as pd
import numpy as np
import requests
import json
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

# Configuração de tema padrão para os gráficos
pio.templates.default = "plotly_white"


In [39]:

# URL dos dados JSON
fonte_dados = 'https://raw.githubusercontent.com/ingridcristh/challenge2-data-science/refs/heads/main/TelecomX_Data.json'

# Requisição HTTP para pegar os dados
resposta = requests.get(fonte_dados)
resposta.raise_for_status()  # Verificar possíveis erros de requisição

# Convertendo o JSON em dicionário Python
dados_cliente = json.loads(resposta.text)

# Normalização das colunas aninhadas
df_clientes = pd.json_normalize(dados_cliente, sep='_')

# Visão geral do dataframe
df_clientes.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   customerID                 7267 non-null   object 
 1   Churn                      7267 non-null   object 
 2   customer_gender            7267 non-null   object 
 3   customer_SeniorCitizen     7267 non-null   int64  
 4   customer_Partner           7267 non-null   object 
 5   customer_Dependents        7267 non-null   object 
 6   customer_tenure            7267 non-null   int64  
 7   phone_PhoneService         7267 non-null   object 
 8   phone_MultipleLines        7267 non-null   object 
 9   internet_InternetService   7267 non-null   object 
 10  internet_OnlineSecurity    7267 non-null   object 
 11  internet_OnlineBackup      7267 non-null   object 
 12  internet_DeviceProtection  7267 non-null   object 
 13  internet_TechSupport       7267 non-null   objec

In [40]:

# Checagem de valores únicos por coluna
for atributo in df_clientes.columns:
    print(f"Coluna: {atributo} - Quantidade de valores únicos: {df_clientes[atributo].nunique()}")
    if df_clientes[atributo].nunique() < 50:
        print(df_clientes[atributo].unique())
        print('-' * 40)


Coluna: customerID - Quantidade de valores únicos: 7267
Coluna: Churn - Quantidade de valores únicos: 3
['No' 'Yes' '']
----------------------------------------
Coluna: customer_gender - Quantidade de valores únicos: 2
['Female' 'Male']
----------------------------------------
Coluna: customer_SeniorCitizen - Quantidade de valores únicos: 2
[0 1]
----------------------------------------
Coluna: customer_Partner - Quantidade de valores únicos: 2
['Yes' 'No']
----------------------------------------
Coluna: customer_Dependents - Quantidade de valores únicos: 2
['Yes' 'No']
----------------------------------------
Coluna: customer_tenure - Quantidade de valores únicos: 73
Coluna: phone_PhoneService - Quantidade de valores únicos: 2
['Yes' 'No']
----------------------------------------
Coluna: phone_MultipleLines - Quantidade de valores únicos: 3
['No' 'Yes' 'No phone service']
----------------------------------------
Coluna: internet_InternetService - Quantidade de valores únicos: 3
['DSL

In [41]:

# Verificando linhas duplicadas
total_duplicados = df_clientes.duplicated().sum()
print(f"Total de duplicados: {total_duplicados}")

# Analisando nulos
print(df_clientes.isnull().sum())

# Verificando campos vazios (strings em branco)
vazios = df_clientes.apply(lambda x: x.astype(str).str.strip() == '').sum()
print(vazios[vazios > 0])


Total de duplicados: 0
customerID                   0
Churn                        0
customer_gender              0
customer_SeniorCitizen       0
customer_Partner             0
customer_Dependents          0
customer_tenure              0
phone_PhoneService           0
phone_MultipleLines          0
internet_InternetService     0
internet_OnlineSecurity      0
internet_OnlineBackup        0
internet_DeviceProtection    0
internet_TechSupport         0
internet_StreamingTV         0
internet_StreamingMovies     0
account_Contract             0
account_PaperlessBilling     0
account_PaymentMethod        0
account_Charges_Monthly      0
account_Charges_Total        0
dtype: int64
Churn                    224
account_Charges_Total     11
dtype: int64


#🔧 Transformação

In [42]:

# Substituindo valores vazios na coluna de cobranças totais
df_clientes['account_Charges_Total'] = df_clientes['account_Charges_Total'].replace(' ', 0).astype(np.float64)

# Convertendo colunas textuais para minúsculo
colunas_texto = df_clientes.select_dtypes(include='object').columns.drop('customerID')
for coluna in colunas_texto:
    df_clientes[coluna] = df_clientes[coluna].str.lower()

# Eliminando registros com evasao em branco
df_clientes = df_clientes[df_clientes['Churn'].str.strip() != '']
print(f"Total de registros após limpeza: {len(df_clientes)}")


Total de registros após limpeza: 7043


In [43]:

# Novo nome das colunas
mapeamento_colunas = {
    'customerID': 'id_cliente',
    'Churn': 'evasao',
    'customer_gender': 'sexo',
    'customer_SeniorCitizen': 'categoria_idoso',
    'customer_Partner': 'tem_parceiro',
    'customer_Dependents': 'tem_dependentes',
    'customer_tenure': 'tempo_contrato',
    'phone_PhoneService': 'telefone_ativo',
    'phone_MultipleLines': 'multiplas_linhas',
    'internet_InternetService': 'tipo_internet',
    'internet_OnlineSecurity': 'seguranca_online',
    'internet_OnlineBackup': 'backup_nuvem',
    'internet_DeviceProtection': 'protecao_aparelho',
    'internet_TechSupport': 'suporte_online',
    'internet_StreamingTV': 'tv_stream',
    'internet_StreamingMovies': 'filmes_stream',
    'account_Contract': 'modelo_contrato',
    'account_PaperlessBilling': 'fatura_online',
    'account_PaymentMethod': 'pagamento',
    'account_Charges_Monthly': 'mensalidade',
    'account_Charges_Total': 'total_gasto'
}

df_clientes = df_clientes.rename(columns=mapeamento_colunas)


In [44]:

# Dicionário de tradução de valores
traducao_valores = {
    'female': 'mulher',
    'male': 'homem',
    'yes': 'sim',
    'no': 'nao',
    'fiber optic': 'fibra',
    'no phone service': 'sem_telefone',
    'no internet service': 'sem_internet',
    'one year': 'anual',
    'month-to-month': 'mensal',
    'two year': 'bienal',
    'mailed check': 'cheque_correio',
    'electronic check': 'cheque_online',
    'credit card (automatic)': 'cartao_auto',
    'bank transfer (automatic)': 'transferencia_auto'
}

# Aplicando tradução em todas as colunas de texto
for col in df_clientes.columns:
    if df_clientes[col].dtype == 'object':
        df_clientes[col] = df_clientes[col].replace(traducao_valores)


In [45]:

# Nova coluna: valor diário
df_clientes['media_diaria'] = ((df_clientes['total_gasto'] / df_clientes['tempo_contrato']) / 30).round(2)
df_clientes['mensalidade'] = df_clientes['mensalidade'].round(2)

# Reposicionando a coluna
colunas = df_clientes.columns.tolist()
colunas.insert(19, colunas.pop(colunas.index('media_diaria')))
df_clientes = df_clientes[colunas]

# Ajustando coluna de idoso
df_clientes['categoria_idoso'] = df_clientes['categoria_idoso'].map({1: 'sim', 0: 'nao'})


#📊 Análise Descritiva + Gráficos + Explicações

###1️⃣ Análise Descritiva Geral (Média, Mediana, Desvio Padrão, etc)

In [46]:
# Análise Descritiva com os nomes de colunas reais
estatisticas = df_clientes[['tempo_contrato', 'mensalidade', 'total_gasto', 'media_diaria']].describe()
print(estatisticas)

# Gráfico Boxplot para as variáveis numéricas
fig = px.box(df_clientes, y=['tempo_contrato', 'mensalidade', 'total_gasto', 'media_diaria'],
             points='all', title='📊 Distribuição das Variáveis Numéricas')
fig.show()


       tempo_contrato  mensalidade  total_gasto  media_diaria
count     7043.000000  7043.000000  7043.000000   7032.000000
mean        32.371149    64.761692  2279.734304      2.159879
std         24.559481    30.090047  2266.794470      1.006217
min          0.000000    18.250000     0.000000      0.460000
25%          9.000000    35.500000   398.550000      1.207500
50%         29.000000    70.350000  1394.550000      2.350000
75%         55.000000    89.850000  3786.600000      3.010000
max         72.000000   118.750000  8684.800000      4.050000


✅ Explicação:
Esse boxplot mostra como estão distribuídos os valores das principais variáveis financeiras e de tempo.
Dá pra ver outliers (valores muito altos) e também a mediana de cada variável.
Exemplo: A cobranca_total tem muitos clientes gastando pouco, mas alguns com gasto muito alto.

###2️⃣ Distribuição da Evasão (Churn)



In [47]:
# Gráfico de Pizza - Distribuição da Evasão
fig = px.pie(df_clientes, names='evasao',
             color='evasao',
             color_discrete_map={'sim': 'red', 'nao': 'green'},
             title='📌 Distribuição da Evasão (Clientes que ficaram vs saíram)')
fig.show()

# Contagem simples de proporção
print(df_clientes['evasao'].value_counts(normalize=True) * 100)


evasao
nao    73.463013
sim    26.536987
Name: proportion, dtype: float64


✅ Explicação:
Visualização percentual de quantos clientes saíram (churn) e quantos permaneceram.

###3️⃣ Contagem de Evasão por Variáveis Categóricas (Gênero, Tipo de Contrato, Método de Pagamento)

In [48]:
# Evasão por Sexo
fig = px.histogram(df_clientes, x='sexo', color='evasao',
                   barmode='group', barnorm='percent',
                   title='🎯 Evasão por Sexo')
fig.show()

# Evasão por Tipo de Contrato
fig = px.histogram(df_clientes, x='modelo_contrato', color='evasao',
                   barmode='group', barnorm='percent',
                   title='🎯 Evasão por Tipo de Contrato')
fig.show()

# Evasão por Método de Pagamento
fig = px.histogram(df_clientes, x='pagamento', color='evasao',
                   barmode='group', barnorm='percent',
                   title='🎯 Evasão por Método de Pagamento')
fig.show()


✅ Explicação:
Mostra o percentual de clientes que cancelaram em cada grupo de gênero, contrato e forma de pagamento.

###4️⃣ Evasão por Variáveis Numéricas (Tempo de Contrato x Total Gasto)

In [49]:
# Scatter Plot: Tempo de Contrato vs Total Gasto
fig = px.scatter(df_clientes, x='tempo_contrato', y='total_gasto', color='evasao',
                 title='📈 Tempo de Contrato vs Total Gasto x Evasão')
fig.show()

# Histograma: Tempo de Contrato por Evasão
fig = px.histogram(df_clientes, x='tempo_contrato', color='evasao',
                   barmode='group', barnorm='percent',
                   title='📊 Distribuição de Evasão por Tempo de Contrato')
fig.show()


✅ Explicação:
Mostra se os clientes com menos tempo de contrato ou com gasto baixo/alto têm mais chance de cancelar.

#📄 Relatório Final - Análise de Evasão de Clientes (Churn) - TelecomX


##💁 Introdução
####O objetivo deste trabalho foi realizar uma análise detalhada sobre o problema de evasão de clientes (Churn) na empresa TelecomX. A evasão de clientes representa um grande desafio para empresas de telecomunicações, pois perder clientes impacta diretamente na receita e na sustentabilidade do negócio.

####Por meio desta análise, buscamos entender os fatores que levam os clientes a cancelar os serviços, identificar padrões de comportamento e sugerir estratégias para reduzir a evasão.

## 🧹 Limpeza e Tratamento de Dados
####Antes de partir para a análise, foi necessário realizar uma etapa de limpeza e preparação dos dados, que incluiu:

####Importação dos dados diretamente de um arquivo JSON hospedado no GitHub.

####Normalização das colunas, uma vez que os dados estavam aninhados em diferentes categorias como "customer", "phone", "internet" e "account".

####Tratamento de valores nulos e vazios, substituindo campos em branco e convertendo colunas numéricas para o tipo correto.

####Padronização dos textos, transformando todas as strings para letras minúsculas.

####Renomeação das colunas para o português, tornando o entendimento mais fácil.

####Tradução dos valores das variáveis categóricas, como “yes” para “sim” e “no” para “não”.

####Criação de novas métricas, como a coluna media_diaria, que representa a média diária de gasto por cliente.



##📌 Análise Exploratória de Dados (EDA)
####Durante a etapa de análise exploratória, realizamos as seguintes investigações:

##📌 Análise Descritiva das Variáveis Numéricas:
####Avaliamos métricas como média, mediana, desvio padrão, valor mínimo e máximo.

####Visualizamos a distribuição das variáveis tempo_contrato, mensalidade, total_gasto e media_diaria com um Boxplot.

####Identificamos que o gasto total e a mensalidade apresentavam alta variabilidade, com presença de outliers.

##📌 Distribuição da Evasão (Churn):
####Criamos um gráfico de pizza para visualizar a proporção de clientes que saíram (churn) e os que permaneceram.

####Observamos que cerca de 26% dos clientes cancelaram os serviços, enquanto 74% permaneceram ativos.

##📌 Evasão por Variáveis Categóricas:
####Analisamos a evasão por gênero, tipo de contrato e método de pagamento:

####Contratos mensais apresentaram a maior taxa de evasão.

####O método de pagamento por cheque eletrônico teve a maior taxa de cancelamento.

####Não houve grande diferença entre gêneros, mas a análise de tipo de contrato e pagamento trouxe insights relevantes.

##📌 Evasão por Variáveis Numéricas:
####Exploramos a relação entre tempo de contrato e gasto total.

####Clientes com poucos meses de contrato apresentaram maior propensão ao cancelamento.

####Também vimos que clientes com gastos mais baixos e com curta duração de contrato são os mais propensos à evasão.

##✅ Conclusões e Insights
###Após a análise, destacamos os principais pontos:

####Clientes com contratos mensais têm maior risco de evasão, mostrando a importância de estratégias de fidelização.

####Formas de pagamento como cheque eletrônico estão altamente associadas à evasão, o que indica uma oportunidade para incentivar outros métodos de pagamento.

####Clientes com baixo tempo de contrato e baixo gasto acumulado têm maior propensão a cancelar.

####Oferecer benefícios nos primeiros meses pode reduzir o churn inicial.

##🔍 Recomendações
###Com base nas análises realizadas, sugerimos as seguintes ações:

####Criar campanhas de incentivo para migrar clientes de contratos mensais para contratos anuais ou bienais.

####Rever a experiência dos clientes que utilizam cheque eletrônico, buscando entender os motivos da alta evasão nesse grupo.

####Investir em um programa de onboarding que melhore a experiência dos clientes nos primeiros meses.

####Oferecer descontos ou benefícios para clientes em risco (ex.: com menos de 6 meses de contrato e baixo gasto mensal).

#### Monitorar continuamente as variáveis de risco, utilizando dashboards com os mesmos indicadores analisados neste projeto.